In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os


In [ ]:
Carga_Datos = pd.read_csv(r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\CSV_Salida.csv")

### Datos temporales para pruebas 


In [ ]:
# datos para las pruebas:
# data = {'Frecuencia (Hz)': [90000000.0, 90000000.0, 90000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 110000000.0, 110000000.0, 110000000.0, 110000000.0, 170000000.0, 170000000.0, 170000000.0, 150000000.0, 150000000.0, 190000000.0, 190000000.0, 190000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0],
#         'dB': [50, 25, 36, 26, 27, 26, 98, 27, 23, 46, 93, 68, 126, 30, 129, 119, 69, 114, 89, 69, 63, 27, 96, 78]}

# data = {
#     'Frecuencia (Hz)': [90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0,
#                         90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 
#                         110000000.0, 110000000.0, 110000000.0, 110000000.0, 
#                         130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0],
#     'dB': [50, 28, 36, 26, 20, 22, 36, 26,
#            27, 23, 46, 93, 27, 23, 46, 93, 
#            26, 27, 26, 98, 
#            27, 23, 46, 93, 27, 23, 46, 93]
# }

data = {
    'Frecuencia (Hz)': [90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0,
                        90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 
                        110000000.0, 110000000.0, 110000000.0, 110000000.0,
                        130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0,
                        90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 
                        110000000.0, 110000000.0, 110000000.0, 110000000.0 ],

    'dB': [50, 28, 36, 26, 20, 22, 36, 26,
           27, 23, 46, 93, 27, 23, 46, 93, 
           26, 27, 26, 27,
           27, 23, 46, 93, 27, 23, 46, 93,
           27, 23, 46, 93, 27, 23, 46, 93,
           27, 23, 46, 93]
}

Carga_Datos = pd.DataFrame(data)
print(Carga_Datos)

###  Borra los primeros datos con caracteres vacíos o nulos  

|Frecuencia (Hz)|        dB        |
|---------------|------------------|
|    nan        |-101.9620361328125|


In [ ]:
Carga_Datos= Carga_Datos.dropna(subset=['Frecuencia (Hz)'])
# print(Carga_Datos['Frecuencia (Hz)'].mean())

In [ ]:
Carga_Datos = Carga_Datos.replace(-379.29779052734375, np.nan)

### Cantidad total de datos por cada frecuencia     


In [ ]:
conteo_frecuencia = Carga_Datos['Frecuencia (Hz)'].value_counts()
print(conteo_frecuencia)

### Separación de muestras por cada frecuencia  

In [ ]:
recuentos_por_muestra = {}

# Convertir la columna 'Frecuencia (Hz)' en un arreglo NumPy
frecuencias = Carga_Datos['Frecuencia (Hz)'].values

# Inicializar variables para llevar el seguimiento de la muestra actual y el conteo actual
muestra_actual = None
conteo_actual = 0

for frecuencia in frecuencias:
    if muestra_actual is None:
        muestra_actual = frecuencia
        conteo_actual = 1
    elif muestra_actual == frecuencia:
        conteo_actual += 1
    else:
        if muestra_actual not in recuentos_por_muestra:
            recuentos_por_muestra[muestra_actual] = []
        recuentos_por_muestra[muestra_actual].append(conteo_actual)
        
        muestra_actual = frecuencia
        conteo_actual = 1

# Agregar el último conjunto de conteos a la salida
if muestra_actual is not None:
    if muestra_actual not in recuentos_por_muestra:
        recuentos_por_muestra[muestra_actual] = []
    recuentos_por_muestra[muestra_actual].append(conteo_actual)

for muestra, conteos in recuentos_por_muestra.items():
    print(f'Frecuencia: {muestra} - Conteos: {", ".join(map(str, conteos))}')

#  Tratado de datos 

### --------------------------------------------------------------------------------------------------------------------------------------------------------

Para poder tratar los datos se dividen por grupos esos hacen referencia a las muestras que se toman, para posteriormente sacar el promedio de estos y el los máximos 

In [ ]:
def dividir_y_guardar_grupos(Carga_Datos, carpeta_salida):
    grupos = []
    grupo_actual = None
    cambios = np.where(Carga_Datos['Frecuencia (Hz)'].values[:-1] > Carga_Datos['Frecuencia (Hz)'].values[1:])[0]

    inicio = 0
    for cambio in cambios:
        grupo_actual = Carga_Datos.iloc[inicio:cambio + 1]
        grupos.append(grupo_actual)
        inicio = cambio + 1

    grupo_actual = Carga_Datos.iloc[inicio:]
    grupos.append(grupo_actual)

    # Crear la carpeta de salida si no existe
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)

    # Guardar los grupos en archivos CSV
    for i, grupo in enumerate(grupos):
        nombre_archivo = os.path.join(carpeta_salida, f'Muestra_{i + 1}.csv')
        grupo.to_csv(nombre_archivo, index=False)
        print(f"Grupo {i + 1} guardado en '{nombre_archivo}'")

carpeta_muestras = 'Muestras'
dividir_y_guardar_grupos(Carga_Datos, carpeta_muestras)


# Tratado de las muestras 

Asignación de las frecuencias para cada muestra, teniendo en cuenta el ancho de banda 

Frec_max-Frec_min/cont_frec-1


In [ ]:
carpeta = r'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras_c'  
archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]

BW_TEMP = 0
MST_ANT = False

# Procesar cada archivo CSV en la carpetax
for archivo_csv in archivos_csv:
    archivo_path = os.path.join(carpeta, archivo_csv)
    data = pd.read_csv(archivo_path)

    # Identificar las secuencias de frecuencias iguales
    secuencias = data['Frecuencia (Hz)'].unique()

    # Calcular el valor de BW
    if len(secuencias) >= 2: # REVISA SI HAY OTRA FRECUENCIA 
        BW = (secuencias[1] - secuencias[0])
        BW_TEMP = BW
        MST_ANT = True
        
    elif (MST_ANT == True):
        BW = BW_TEMP

    else:               # si no no hubo una muestra anterior y no tiene un valor de BW lo tomara por defecto como 20M.
        BW = 10000000

    # Crear un nuevo DataFrame para los resultados
    nuevo_df = pd.DataFrame(columns=['Frecuencia (Hz)', 'dB'])

    # Calcular y agregar los nuevos valores de frecuencia 
    for secuencia in secuencias:
        num_samples = (data['Frecuencia (Hz)'] == secuencia).sum()
        
        min_value = secuencia - (BW / 2)
        max_value = secuencia + (BW / 2)

        # Asegurar que num_samples sea al menos 2
        if num_samples == 1:
            num_samples = 2

        incremento_frec = BW / (num_samples - 1)
        nuevos_valores_frecuencia = min_value + np.arange(num_samples) * incremento_frec
        nuevos_valores_frecuencia = pd.Series(nuevos_valores_frecuencia)
        nuevos_valores_frecuencia.name = 'Frecuencia (Hz)'
        nuevo_df = pd.concat([nuevo_df, pd.concat([nuevos_valores_frecuencia, data[data['Frecuencia (Hz)'] == secuencia]['dB'].reset_index(drop=True)], axis=1)], ignore_index=True)

    # Ordenar el nuevo DataFrame por frecuencia
    nuevo_df.sort_values(by=['Frecuencia (Hz)'], inplace=True)

    # Reiniciar los índices del nuevo DataFrame
    nuevo_df.reset_index(drop=True, inplace=True)

    # Sobreescribir el archivo CSV original con los nuevos datos
    nuevo_df.to_csv(archivo_path, index=False)

    print(f'Archivo procesado y sobrescrito: {archivo_path}')
    print (f"BW: {BW} ")


# Filtrado de datos 

Promedio y máximos unificado de todas las muestras 

In [ ]:

# Ruta de la carpeta de muestras
carpeta_archivos = r'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras_c'

# Obtiene la ruta del directorio anterior
directorio= os.path.dirname(carpeta_archivos)

# Carpeta para guardar los resultados
carpeta_resultados = os.path.join(directorio, 'Resultados')

# Verifica si la carpeta ya existe, y si no, créala
if not os.path.exists(carpeta_resultados):
    os.makedirs(carpeta_resultados)

# Tolerancia para realizar el análisis de los datos 
tolerancia = 1

# Función para procesar un archivo CSV y obtener el promedio de dB dentro de la tolerancia
def procesar_archivo(archivo):
    data = pd.read_csv(archivo)
    frecuencia = data['Frecuencia (Hz)'].values
    db = data['dB'].values
    promedios = []
    maximos = []
    

    inicio = 0
    fin = 0
    
    for i in range(len(frecuencia)):
        if frecuencia[i] <= frecuencia[inicio] + tolerancia:
            fin = i
        else:
            # Calcula el promedio y máximo dentro de la tolerancia
            promedio = np.mean(db[inicio:fin+1])
            maximo = np.max(db[inicio:fin+1])
            promedios.append([frecuencia[inicio], promedio])
            maximos.append([frecuencia[inicio], maximo])
            
            # Actualiza el inicio y fin para el siguiente rango
            inicio = i
            fin = i
    
    # Agrega el último rango
    promedio = np.mean(db[inicio:])
    maximo = np.max(db[inicio:])
    promedios.append([frecuencia[inicio], promedio])
    maximos.append([frecuencia[inicio], maximo])
    
    return pd.DataFrame(promedios, columns=['Frecuencia (Hz)', 'Promedio (dB)']), pd.DataFrame(maximos, columns=['Frecuencia (Hz)', 'Máximo (dB)'])

# Obtiene la lista de archivos en la carpeta
archivos = [os.path.join(carpeta_archivos, archivo) for archivo in os.listdir(carpeta_archivos) if archivo.endswith('.csv')]
archivos.sort()

promedio_acumulado = pd.DataFrame(columns=['Frecuencia (Hz)', 'Promedio (dB)'])
maximo_acumulado = pd.DataFrame(columns=['Frecuencia (Hz)', 'Máximo (dB)'])

for archivo in archivos[:-1]:
    promedio, maximo = procesar_archivo(archivo)
    
    # Combina los resultados con el promedio acumulado
    promedio_acumulado = pd.concat([promedio_acumulado, promedio], ignore_index=True)
    maximo_acumulado = pd.concat([maximo_acumulado, maximo], ignore_index=True)

# Calcula el promedio final
promedio_final = promedio_acumulado.groupby('Frecuencia (Hz)').mean().reset_index()
maximo_final = maximo_acumulado.groupby('Frecuencia (Hz)').max().reset_index()

# Ruta para guardar los archivos en la carpeta "Resultados"
ruta_promedio_final = os.path.join(carpeta_resultados, 'promedio_final.csv')
ruta_maximo_final = os.path.join(carpeta_resultados, 'maximo_final.csv')

# Guarda los resultados en dos archivos CSV separados en la carpeta "Resultados"
promedio_final.to_csv(ruta_promedio_final, index=False)
maximo_final.to_csv(ruta_maximo_final, index=False)


# Genera los plots de las muestras por separado 

Toma las muestras generadas y gráfica cada una de ellas por grupos de 1000 datos para que sea visibles

In [ ]:
carpeta_datos = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras"

# Crear la carpeta 'plots' si no existe
carpeta_plots = os.path.join(carpeta_datos, "plots")
os.makedirs(carpeta_plots, exist_ok=True)

# Lista de archivos CSV en la carpeta de datos
archivos_csv = [archivo for archivo in os.listdir(carpeta_datos) if archivo.endswith(".csv")]

def dividir_en_grupos(datos):
    grupos = []
    grupo_actual = []
    for i, fila in enumerate(datos):
        grupo_actual.append(fila)
        if (i + 1) % 1000 == 0:
            grupos.append(grupo_actual)
            grupo_actual = []
    if grupo_actual:
        grupos.append(grupo_actual)
    return grupos

# Procesar cada archivo CSV
for archivo_csv in archivos_csv:
    # Leer el archivo CSV
    ruta_csv = os.path.join(carpeta_datos, archivo_csv)
    df = pd.read_csv(ruta_csv)

    # Dividir los datos en grupos de 2000
    grupos = dividir_en_grupos(df.values)

    # Generar y guardar los gráficos para cada grupo
    for i, grupo in enumerate(grupos):
        plt.figure(figsize=(18, 8))
        x = [fila[0] for fila in grupo]
        y = [fila[1] for fila in grupo]
        plt.plot(x, y)
        plt.xlabel("Frecuencia (Hz)")
        plt.ylabel("dB")
        plt.title(f"Gráfico {i + 1} - {archivo_csv}")
        ruta_plot = os.path.join(carpeta_plots, f"{archivo_csv}_{i + 1}.png")
        plt.savefig(ruta_plot)
        plt.close()

print("Gráficos generados y guardados en la carpeta 'plots'.")

# Grafica individual 



In [ ]:
# Leer el archivo CSV con los datos
archivo_csv = r'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras\Muestra_1.csv'
df = pd.read_csv(archivo_csv)

# Extraer las columnas de frecuencia y promedio
frecuencia = df['Frecuencia (Hz)'].to_numpy()
promedio = df['dB'].to_numpy()

plt.figure(figsize=(18, 8))
plt.plot(frecuencia, promedio, marker='', linestyle='-')  

# Configurar etiquetas y título
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Promedio (dB)')
plt.title('Promedio Total por Intervalo de Frecuencia')

# Mostrar el gráfico
plt.xticks(rotation=45)
plt.grid(True)  # Agrega una cuadrícula para mayor claridad
plt.tight_layout()
plt.show()

# Setpoint en base al valor promedio genera csv con valores que max que sobrepasaron el setpoint


### máximos partiendo de setpoint para un único archivo, promedio de todas las muestras        

In [ ]:
def Setpoint_Max(archivo_csv, Setpoint,carpeta_resultados):
    data = pd.read_csv(archivo_csv)
    encabezados = data.columns
    promedio_segunda_columna = data.iloc[:, 1].mean()
    setpoint = promedio_segunda_columna + Setpoint  
    valores_superan_setpoint = data[data.iloc[:, 1] > setpoint]


    nombre_original_sin_extension = os.path.splitext(os.path.basename(archivo_csv))[0]

    nombre_salida = f"{nombre_original_sin_extension}_Setpoint_Max.csv"

    if not os.path.exists(carpeta_resultados):
        os.makedirs(carpeta_resultados)

    ruta_salida = os.path.join(carpeta_resultados, nombre_salida)
    valores_superan_setpoint.to_csv(ruta_salida, index=False)
    
    print(f"Valor promedio dB {nombre_original_sin_extension}: {promedio_segunda_columna} dB")
    print(f"Valor Setpoint para {nombre_original_sin_extension}: {setpoint} dB")
    print(f"El archivo de resultados se ha guardado en: {ruta_salida}")


archivo_csv = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Resultados\promedio_final.csv"  
carpeta_resultados = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Resultados"
Setpoint = 20 

Setpoint_Max(archivo_csv, Setpoint ,carpeta_resultados)


### maximos partiendo de setpoint para toda la carpeta de muestras 

In [ ]:
def procesar_archivos_en_carpeta(carpeta_entrada, carpeta_resultados,Setpoint ):
    archivos_csv = [f for f in os.listdir(carpeta_entrada) if f.endswith('.csv')]
    
    for archivo in archivos_csv:
        archivo_csv = os.path.join(carpeta_entrada, archivo)
        data = pd.read_csv(archivo_csv)
        encabezados = data.columns

        promedio_segunda_columna = data.iloc[:, 1].mean()
        setpoint = promedio_segunda_columna + Setpoint 

        valores_superan_setpoint = data[data.iloc[:, 1] > setpoint]

        nombre_original_sin_extension = os.path.splitext(os.path.basename(archivo_csv))[0]
        nombre_salida = f"{nombre_original_sin_extension}_Setpoint_Max.csv"

        if not os.path.exists(carpeta_resultados):
            os.makedirs(carpeta_resultados)

        ruta_salida = os.path.join(carpeta_resultados, nombre_salida)
        valores_superan_setpoint.to_csv(ruta_salida, index=False)

        print(f"Valor promedio dB {archivo}: {promedio_segunda_columna} dB")
        print(f"Valor Setpoint para {archivo}: {setpoint} dB")
        print(f"El archivo de resultados se ha guardado en: {ruta_salida}")

carpeta_entrada = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras"
carpeta_resultados = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Resultados"
Setpoint = 20 

procesar_archivos_en_carpeta(carpeta_entrada, carpeta_resultados,Setpoint)

# pruebas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargar los datos del archivo CSV en un DataFrame
df = pd.read_csv(R'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras\Muestra_2.csv')

# Seleccionar los primeros 10 datos (ajusta el número según tus necesidades)
df_subset = df.loc[7168:8192]

# Crear el gráfico utilizando los datos seleccionados
plt.figure(figsize=(20, 10))  # Opcional: ajusta el tamaño del gráfico
plt.plot(df_subset['Frecuencia (Hz)'], df_subset['dB'])  # Reemplaza 'ColumnaX' y 'ColumnaY' por los nombres de tus columnas
plt.title('Gráfico de los primeros 10 datos')
plt.xlabel('Eje X')
plt.ylabel('Eje Y')
plt.grid(True)

# Mostrar el gráfico
plt.show()


In [ ]:
def dividir_y_guardar_grupos(Carga_Datos):
    grupos = []
    grupo_actual = None
    cambios = np.where(Carga_Datos['Frecuencia (Hz)'].values[:-1] > Carga_Datos['Frecuencia (Hz)'].values[1:])[0]

    inicio = 0
    for cambio in cambios:
        grupo_actual = Carga_Datos.iloc[inicio:cambio + 1]
        grupos.append(grupo_actual)
        inicio = cambio + 1

    grupo_actual = Carga_Datos.iloc[inicio:]
    grupos.append(grupo_actual)

    return grupos

grupos = dividir_y_guardar_grupos(Carga_Datos)

carpeta_muestras = 'Muestras'
if not os.path.exists(carpeta_muestras):
    os.makedirs(carpeta_muestras)

cantidad_deseada = 4

for i, grupo in enumerate(grupos):
    # Obtener valores únicos de frecuencia en el grupo actual
    valores_de_frecuencia = grupo['Frecuencia (Hz)'].unique()

    # Crear un diccionario de subgrupos para el grupo actual
    subgrupos = {}

    # Iterar a través de los valores de frecuencia
    for frecuencia in valores_de_frecuencia:
        subgrupo = grupo[grupo['Frecuencia (Hz)'] == frecuencia]
        # Asegúrate de que haya suficientes datos para la cantidad deseada
        if len(subgrupo) >= cantidad_deseada:
            # Toma la cantidad deseada de datos
            subgrupo = subgrupo.head(cantidad_deseada)
        subgrupos[frecuencia] = subgrupo

    # Crear un DataFrame con los subgrupos concatenados
    nuevo_dataframe = pd.concat(subgrupos.values(), ignore_index=True)

    # Guardar el nuevo DataFrame en un archivo CSV en la carpeta de muestras
    nombre_archivo = os.path.join(carpeta_muestras, f'Muestra_{i + 1}.csv')
    nuevo_dataframe.to_csv(nombre_archivo, index=False)
    print(f"Grupo {i + 1}: Muestras guardadas en '{nombre_archivo}'")

# Prueba para aprovechar todos los datos y tener menos margen de error 

In [ ]:
import pandas as pd
import numpy as np

data = {
    'Frecuencia (Hz)': [90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0,
                        90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 
                        110000000.0, 110000000.0, 110000000.0, 110000000.0,
                        130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0,
                        90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 
                        110000000.0, 110000000.0, 110000000.0, 110000000.0 ],

    'dB': [50, 28, 36, 26, 20, 22, 36, 26,
           27, 23, 46, 93, 27, 23, 46, 93, 
           26, 27, 26, 27,
           27, 23, 46, 93, 27, 23, 46, 93,
           27, 23, 46, 93, 27, 23, 46, 93,
           27, 23, 46, 93]
}

Carga_Datos = pd.DataFrame(data)

muestras = []
while len(Carga_Datos) > 0:
    frecuencia_inicial = Carga_Datos.iloc[0, 0]
    mascara = Carga_Datos['Frecuencia (Hz)'] == frecuencia_inicial
    muestra = Carga_Datos[mascara].copy()
    muestras.append(muestra)
    # Encuentra el índice donde la frecuencia cambia y divide la muestra
    cambio_frecuencia_idx = muestra[muestra['Frecuencia (Hz)'] != frecuencia_inicial].index
    if not cambio_frecuencia_idx.empty:
        primer_cambio_idx = cambio_frecuencia_idx.min()
        Carga_Datos = Carga_Datos[~mascara]
    else:
        Carga_Datos = pd.DataFrame()  # No hay más muestras

cantidad_deseada = 4  # Cambia esto según tu necesidad

subgrupos_de_frecuencia = []
for muestra in muestras:
    grupos = {}
    for frecuencia, datos in muestra.groupby('Frecuencia (Hz)'):
        divisiones = len(datos) // cantidad_deseada
        subgrupos = [datos[i * cantidad_deseada:(i + 1) * cantidad_deseada] for i in range(divisiones)]
        grupos[frecuencia] = subgrupos
    subgrupos_de_frecuencia.append(grupos)

# Acceder a un subgrupo específico, por ejemplo, el segundo subgrupo de la primera muestra con frecuencia 90000000.0
subgrupo = subgrupos_de_frecuencia[0][90000000.0][5]

print(subgrupo)


codigo funcional para sacar las muestras por valor de frecuencia, debe tenere los valores separados de las frecuencias 

In [ ]:
import pandas as pd

data = {
    'Frecuencia (Hz)': [90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0,
                        90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0],

    'dB': [50, 28, 36, 26, 20, 22, 36, 26,
           27, 23, 46, 93, 27, 23, 46, 93]
}

df = pd.DataFrame(data)


filas_por_grupo = 4

cantidad_grupos = len(data['Frecuencia (Hz)']) // filas_por_grupo

columna_frecuencia = pd.Series(data['Frecuencia (Hz)'][:cantidad_grupos]).repeat(filas_por_grupo).reset_index(drop=True)

df['Grupo'] = (df.index // filas_por_grupo) + 1

df['Nuevo_Indice'] = df.groupby('Grupo').cumcount()

df_pivot = df.pivot(index='Nuevo_Indice', columns='Grupo', values='dB')

df_pivot.columns = [f'dB{col}' for col in df_pivot.columns]

df_pivot.reset_index(drop=True, inplace=True)
df_pivot.index.name = 'Indice'

df_pivot['Frecuencia (Hz)'] = columna_frecuencia

df_pivot = df_pivot[['Frecuencia (Hz)'] + [col for col in df_pivot.columns if col != 'Frecuencia (Hz)']]

df_promedio = df_pivot.copy()
df_promedio['Promedio_dB'] = df_pivot.iloc[:, 1:].mean(axis=1)

print(df_promedio)

df_promedio_frecuencia = df_promedio[['Frecuencia (Hz)', 'Promedio_dB']]

print(df_promedio_frecuencia)


## Separacion de las muestras del dataframe y rimedio de las muestrsa separada por las frecuencias 

In [ ]:
# divide por grupos de frecuencias y de muestras 

import pandas as pd
import numpy as np
import os

def dividir_y_guardar_grupos(Carga_Datos, carpeta_salida):
    grupos = []
    grupo_actual = None
    cambios = np.where(Carga_Datos['Frecuencia (Hz)'].values[:-1] > Carga_Datos['Frecuencia (Hz)'].values[1:])[0]

    inicio = 0
    for cambio in cambios:
        grupo_actual = Carga_Datos.iloc[inicio:cambio + 1]
        grupos.append(grupo_actual)
        inicio = cambio + 1

    grupo_actual = Carga_Datos.iloc[inicio:]
    grupos.append(grupo_actual)

    
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)

    def agrupar_por_frecuencia(df):
        grupos_frecuencia = {}
        for _, row in df.iterrows():
            frecuencia = row['Frecuencia (Hz)']
            if frecuencia not in grupos_frecuencia:
                grupos_frecuencia[frecuencia] = []
            grupos_frecuencia[frecuencia].append(row)
        return list(grupos_frecuencia.values())

    for i, grupo in enumerate(grupos):
        
        grupos_frecuencia = agrupar_por_frecuencia(grupo)

        for j, subgrupo in enumerate(grupos_frecuencia):
            nombre_archivo = os.path.join(carpeta_salida, f'Muestra_{i + 1}_Grupo_{j + 1}.csv')
            df = pd.DataFrame(subgrupo)
            df.to_csv(nombre_archivo, index=False)
            print(f"Grupo {i + 1}, Subgrupo {j + 1} guardado en '{nombre_archivo}'")

carpeta_muestras = 'Muestras'
dividir_y_guardar_grupos(Carga_Datos, carpeta_muestras)

In [ ]:
# transpone los datos cada 1024 datos y saca el promedio de este en la ultima columna 

carpeta_entrada = r'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras'


filas_por_grupo = 1024


archivos = os.listdir(carpeta_entrada)

for archivo in archivos:
    if archivo.endswith('.csv'):  
        
        data = pd.read_csv(os.path.join(carpeta_entrada, archivo))

        cantidad_grupos = len(data['Frecuencia (Hz)']) // filas_por_grupo
        columna_frecuencia = pd.Series(data['Frecuencia (Hz)'][:cantidad_grupos]).repeat(filas_por_grupo).reset_index(drop=True)
        data['Grupo'] = (data.index // filas_por_grupo) + 1
        data['Nuevo_Indice'] = data.groupby('Grupo').cumcount()
        df_pivot = data.pivot(index='Nuevo_Indice', columns='Grupo', values='dB')
        df_pivot.columns = [f'dB{col}' for col in df_pivot.columns]
        df_pivot.reset_index(drop=True, inplace=True)
        df_pivot.index.name = 'Indice'
        df_pivot['Frecuencia (Hz)'] = columna_frecuencia
        df_pivot = df_pivot[['Frecuencia (Hz)'] + [col for col in df_pivot.columns if col != 'Frecuencia (Hz)']]
        df_promedio = df_pivot.copy()
        df_promedio['Promedio_dB'] = df_pivot.iloc[:, 1:].mean(axis=1)

        #df_promedio = df_promedio.iloc[:, [0, -1]]

        df_promedio.to_csv(os.path.join(carpeta_entrada, archivo), index=False)

        print(f"Archivo '{archivo}' procesado y sobrescrito.")

print("Proceso de procesamiento y sobrescritura de grupos completado.")


In [ ]:
# elimimna los datos restantes, concerva primera columna (Frecuencia) y ultima columna, promedio de frecuencias por muestra 

import os
import pandas as pd

# Ruta de la carpeta que contiene los archivos CSV
carpeta = r'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras'

# Enumera todos los archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]

for archivo in archivos_csv:
    ruta_completa = os.path.join(carpeta, archivo)

    # Lee el DataFrame desde el archivo CSV
    df = pd.read_csv(ruta_completa)

    # Conserva solo la primera y la última columna
    df = df.iloc[:, [0, -1]]

    # Renombra la última columna como "dB"
    df = df.rename(columns={df.columns[-1]: 'dB'})

    # Sobrescribe el archivo CSV original con los cambios
    df.to_csv(ruta_completa, index=False)


In [ ]:
# concatena lso datos que sean de la misma muestra es decir crea un solo archivo por muestra 

import os
import pandas as pd

directorio = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras"

dataframes_por_muestra = {}

for archivo in os.listdir(directorio):
    if archivo.endswith(".csv"):
        
        nombre_muestra = archivo.split("_")[1]

        df = pd.read_csv(os.path.join(directorio, archivo))

        if nombre_muestra in dataframes_por_muestra:
            dataframes_por_muestra[nombre_muestra] = pd.concat([dataframes_por_muestra[nombre_muestra], df], ignore_index=True)
        else:
            dataframes_por_muestra[nombre_muestra] = df

for nombre_muestra, df in dataframes_por_muestra.items():
    nombre_archivo_salida = f"{nombre_muestra}.csv"
    df.to_csv(nombre_archivo_salida, index=False)


In [ ]:
carpeta = r'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras_c'  
archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]

BW_TEMP = 0
MST_ANT = False

# Procesar cada archivo CSV en la carpetax
for archivo_csv in archivos_csv:
    archivo_path = os.path.join(carpeta, archivo_csv)
    data = pd.read_csv(archivo_path)

    # Identificar las secuencias de frecuencias iguales
    secuencias = data['Frecuencia (Hz)'].unique()

    # Calcular el valor de BW
    if len(secuencias) >= 2: # REVISA SI HAY OTRA FRECUENCIA 
        BW = (secuencias[1] - secuencias[0])
        BW_TEMP = BW
        MST_ANT = True
        
    elif (MST_ANT == True):
        BW = BW_TEMP

    else:               # si no no hubo una muestra anterior y no tiene un valor de BW lo tomara por defecto como 20M.
        BW = 10000000

    # Crear un nuevo DataFrame para los resultados
    nuevo_df = pd.DataFrame(columns=['Frecuencia (Hz)', 'dB'])

    # Calcular y agregar los nuevos valores de frecuencia 
    for secuencia in secuencias:
        num_samples = (data['Frecuencia (Hz)'] == secuencia).sum()
        
        min_value = secuencia - (BW / 2)
        max_value = secuencia + (BW / 2)

        # Asegurar que num_samples sea al menos 2
        if num_samples == 1:
            num_samples = 2

        incremento_frec = BW / (num_samples - 1)
        nuevos_valores_frecuencia = min_value + np.arange(num_samples) * incremento_frec
        nuevos_valores_frecuencia = pd.Series(nuevos_valores_frecuencia)
        nuevos_valores_frecuencia.name = 'Frecuencia (Hz)'
        nuevo_df = pd.concat([nuevo_df, pd.concat([nuevos_valores_frecuencia, data[data['Frecuencia (Hz)'] == secuencia]['dB'].reset_index(drop=True)], axis=1)], ignore_index=True)

    # Ordenar el nuevo DataFrame por frecuencia
    nuevo_df.sort_values(by=['Frecuencia (Hz)'], inplace=True)

    # Reiniciar los índices del nuevo DataFrame
    nuevo_df.reset_index(drop=True, inplace=True)

    # Sobreescribir el archivo CSV original con los nuevos datos
    nuevo_df.to_csv(archivo_path, index=False)

    print(f'Archivo procesado y sobrescrito: {archivo_path}')
    print (f"BW: {BW} ")
